In [1]:
import pandas as pd
import numpy as np
import scipy.stats
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [3]:
df_geo = pd.read_csv('/project/processed_data/grouped_bymean_data.csv')

In [4]:
df_geo.head()

,geometry,Storefront,Visible Sign of Business,Business WiFi,Business tables,Business Plugs,Business Internal CCTV,External CCTV,Number of female employees,Accepts credit card,Incline,Width,Ramp conditions,Ramp // Crosswalk,Ramp obstacles,Crosswalk width,Vertical signs,Crossing length,Access to people with disabilities,Broken public property,Incivility,Obstacles,Physical aspects in poor condition,Poor infrastructure,Positive elements,Broken drain-cover,Drug addicts,Federal Judiciary,Panic button,Vandalism,graffiti,homelessness,inclined ramp,indication of security,mobile vendors,police presence,risky building,risky pavement,risky road,tourist map,trash,vehicle abandoned,excellent street lighting,good street lighting,regular street lighting,Coffee Shop,Food joint,Ice Cream Shop,Lighting Stores,Restaurant,Open-Weekdays,Open-Weekends,fully painted crosswalk,not painted crosswalk,partially painted crosswalk,Public WiFi,number of bikes,Mean Inhabitants,Employment,Crime Degree,school_count,Flooding,hospitals_count,Deaths,Injuries,Collision,Run over,Metro,Metrobus,Trolleybus,RTP,Concessioned,e-bike,Total coverage of transport
0,POLYGON ((483895.6627641306 2147977.5123609374...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.333333,2.0,1.666667,0.666667,1.0,2.0,0.0,0.666667,0.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,1.0,0.0,0.596230,2.0,1.5,16.0,5.0,64.0,0.0,1.0,0.0,1.5,1.0,1.0,0.0,1.0,1.0,0.5,4.5
1,"POLYGON ((483983.9277624827 2148086.839068036,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,36.0,0.636282,2.0,1.0,1.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.5,4.5
2,"POLYGON ((484015.5864170455 2148025.338677514,...",1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,1.0,0.5,0.000000,0.0,0.000000,0.0,30.0,0.640760,2.0,1.0,8.0,5.0,16.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0
3,POLYGON ((484083.10588954366 2148038.433536383...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.0,1.0,0.5,0.000000,0.0,0.000000,0.0,30.0,0.640760,1.5,1.0,16.0,5.0,32.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0
4,POLYGON ((484066.97255161795 2148144.214641084...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,30.0,0.636282,1.5,1.0,6.0,5.0,6.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.5,1.0,1.0,4.5


We will drop the geometry column for further processing.

In [5]:
df=df_geo.drop('geometry',axis=1)

In [6]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Unique values in column '{column}':")
    print(unique_values)
    print()


Unique values in column 'Storefront':
[0. 1.]

Unique values in column 'Visible Sign of Business':
[0. 1.]

Unique values in column 'Business WiFi':
[0.        1.        0.8       0.9047619]

Unique values in column 'Business tables':
[0. 1.]

Unique values in column 'Business Plugs':
[0. 1.]

Unique values in column 'Business Internal CCTV':
[0. 1.]

Unique values in column 'External CCTV':
[0.  1.  0.8]

Unique values in column 'Number of female employees':
[ 0.          2.          2.25        1.          1.33333333  2.5
  3.5         3.         10.          0.5         7.          1.2
  4.          4.5         5.          3.6         4.19047619  2.4       ]

Unique values in column 'Accepts credit card':
[0.   0.5  0.75]

Unique values in column 'Incline':
[1.33333333 0.         1.         1.5        2.         1.2
 1.28571429]

Unique values in column 'Width':
[2.         0.         1.66666667 1.5        1.         1.9
 1.85714286]

Unique values in column 'Ramp conditions':
[1.66

In [7]:
column_means = df.mean()
column_means

Storefront                             0.438972
Visible Sign of Business               0.438972
Business WiFi                          0.425533
Business tables                        0.434690
Business Plugs                         0.438972
Business Internal CCTV                 0.426124
External CCTV                          0.435974
Number of female employees             1.000148
Accepts credit card                    0.035867
Incline                                0.512022
Width                                  0.916723
Ramp conditions                        0.930142
Ramp // Crosswalk                      0.456164
Ramp obstacles                         0.452055
Crosswalk width                        0.779566
Vertical signs                         0.046467
Crossing length                        0.285296
Access to people with disabilities     0.217538
Broken public property                 0.008565
Incivility                             0.087794
Obstacles                              0

# Trial 1: K-Means Clustering to get the centroids for each category with Standardization

K-means is an unsupervised learning algorithm used to partition a dataset into K distinct, non-overlapping subsets (or clusters) based on the data points' features. The way K-means works is by assigning each data point to the cluster whose mean (also called centroid or center) is nearest. The means are calculated by taking the average of all the data points in the cluster. One important thing to note is the process of feature scaling before applying K-means clustering. Since K-means clustering relies on the Euclidean distance between points, features with larger scales can disproportionately influence the clustering results. So, we will scale our data to have zero mean and unit variance before applying K-means clustering. 

In [8]:
# Initialize a new scaler instance
scaler = StandardScaler()

# Fit the scaler and transform the data
df_scaled = scaler.fit_transform(df)

In [9]:
# Set number of clusters
# 3 for our 3 categories: A - Desirable, B - Acceptable, C - Inadequate
k = 3

# Create a KMeans instance with k clusters
kmeans = KMeans(n_clusters=k, random_state=0)

# Fit the model to your scaled data
kmeans.fit(df_scaled)

# Get the coordinates of the cluster centers in the scaled space
scaled_centroids = kmeans.cluster_centers_

KMeans(n_clusters=3, random_state=0)

In [10]:
# Transform the scaled centroids back to the original feature space
original_centroids = scaler.inverse_transform(scaled_centroids)

The values for the centroids found by K-means using standardization do not follow the trends of the original data. This suggests that standardization may not have been suitable for this task. We will implement min-max normalization to test if it performs better.

In [11]:
original_centroids[0]

array([-2.77555756e-16, -2.77555756e-16,  7.77156117e-16, -5.55111512e-16,
       -2.77555756e-16,  2.77555756e-16, -2.22044605e-16, -1.33226763e-15,
        3.46944695e-17, -5.55111512e-16, -1.88737914e-15,  3.33066907e-16,
        6.66133815e-16,  3.88578059e-16,  1.33226763e-15, -6.93889390e-18,
       -4.44089210e-16,  2.22044605e-16,  6.28930818e-03,  1.32075472e-01,
        1.63522013e-01,  2.01257862e-01,  1.50943396e-01,  3.52201258e-01,
        1.25786164e-01,  2.51572327e-02, -2.60208521e-18,  1.88679245e-01,
       -5.20417043e-18,  1.32075472e-01,  8.17610063e-02,  2.60208521e-18,
        4.33680869e-18,  6.91823899e-02,  2.51572327e-02,  1.19496855e-01,
        6.28930818e-02,  8.17610063e-02,  7.54716981e-02,  2.51572327e-02,
        4.33680869e-18,  3.14465409e-02,  1.88679245e-02,  6.28930818e-03,
       -1.38777878e-16, -1.66533454e-16, -1.30104261e-18,  1.24900090e-16,
        8.32667268e-17, -5.55111512e-17,  3.33066907e-16, -4.44089210e-16,
        4.16333634e-17, -

In [12]:
np.array(df.iloc[40])

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        2.        ,  2.        ,  1.        ,  1.        ,  2.        ,
        0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.2       ,
        0.8       ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.2       ,  1.        ,  0.        ,  0.        ,  1.        ,
       30.        ,  0.64076007,  2.5       ,  1.        , 24.        ,
        5.        , 48.        ,  0.        ,  1.        ,  0.        ,
        2.        ,  1.        ,  1.        ,  0.        ,  1.  

# Trial 2: K-Means Clustering to get the centroids for each category with Min-Max Normalization

We perform this operation on numerical columns, as it does not make sense for binary or categorical data.

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Set numerical columns
df_columns_to_normalize = ['Number of female employees', 'Incline','Width','Ramp conditions',
                           'Crosswalk width','Public WiFi', 'number of bikes', 
                           'Employment','Broken public property',
                           'Crime Degree', 'school_count', 'hospitals_count',
                           'Injuries', 'Collision', 'Run over', 'Total coverage of transport']

n_scaler = MinMaxScaler()

# Create a copy of DataFrame
df_normalized = df.copy()

df_normalized[df_columns_to_normalize] = n_scaler.fit_transform(df_normalized[df_columns_to_normalize])


In [14]:
# Set number of clusters
# 3 for our 3 categories: A - Desirable, B - Acceptable, C - Inadequate
k = 3

# Create a KMeans instance with k clusters
kmeans_n = KMeans(n_clusters=k, random_state=0)

# Fit the model to your scaled data
kmeans_n.fit(df_normalized)

# Get the coordinates of the cluster centers in the scaled space
scaled_centroids_n = kmeans_n.cluster_centers_

KMeans(n_clusters=3, random_state=0)

In [17]:
# Get the indices of the columns you normalized in the original dataframe
column_indices = [df.columns.get_loc(col) for col in df_columns_to_normalize]

# Select only the centroids for these columns
scaled_centroids_subset = scaled_centroids_n[:, column_indices]

# Now you can apply inverse_transform
original_centroids_n = n_scaler.inverse_transform(scaled_centroids_subset)


In [20]:
original_centroids_n[0]
scaled_centroids_subset[0]
scaled_centroids_n[0]

array([-1.11022302e-15,  9.99200722e-16, -1.33226763e-15, -1.33226763e-15,
       -1.22124533e-15,  1.17295597e+00,  2.43270440e+01,  2.94025157e+00,
        6.28930818e-03,  1.75262055e+00,  8.37106918e+00,  1.45220126e+01,
        2.57861635e-01,  9.10729825e-18,  3.11320755e-01,  4.58176101e+00])

array([-1.11022302e-16,  4.99600361e-16, -6.66133815e-16, -6.66133815e-16,
       -6.10622664e-16,  1.46619497e-01,  6.75751223e-01,  4.85062893e-01,
        3.14465409e-03,  3.01048218e-01,  1.17001098e-01,  5.30275003e-02,
        5.15723270e-02,  3.03576608e-18,  1.03773585e-01,  7.63626834e-01])

array([ 7.21644966e-16,  7.21644966e-16,  4.99600361e-16,  4.99600361e-16,
        7.21644966e-16,  4.99600361e-16, -4.99600361e-16, -1.11022302e-16,
       -4.85722573e-17,  4.99600361e-16, -6.66133815e-16, -6.66133815e-16,
        2.77555756e-16,  6.66133815e-16, -6.10622664e-16,  0.00000000e+00,
       -3.33066907e-16,  1.94289029e-16,  3.14465409e-03,  1.32075472e-01,
        1.63522013e-01,  2.01257862e-01,  1.50943396e-01,  3.52201258e-01,
        1.25786164e-01,  2.51572327e-02,  3.03576608e-18,  1.88679245e-01,
        1.21430643e-17,  1.32075472e-01,  8.17610063e-02,  3.03576608e-18,
        6.07153217e-18,  6.91823899e-02,  2.51572327e-02,  1.19496855e-01,
        6.28930818e-02,  8.17610063e-02,  7.54716981e-02,  2.51572327e-02,
        3.03576608e-18,  3.14465409e-02,  1.88679245e-02,  6.28930818e-03,
       -2.22044605e-16, -1.66533454e-16,  3.03576608e-18,  5.55111512e-17,
       -1.11022302e-16, -5.55111512e-17, -7.21644966e-16, -4.44089210e-16,
        2.77555756e-17,  

# Trial 3: Univariate Percentiles with Normalization

We had already normalized the data using MinMaxScaler in Trial 2. We will use that version of the DataFrame for Trial 3 as well.

In [21]:
df_normalized.head(1)

,Storefront,Visible Sign of Business,Business WiFi,Business tables,Business Plugs,Business Internal CCTV,External CCTV,Number of female employees,Accepts credit card,Incline,Width,Ramp conditions,Ramp // Crosswalk,Ramp obstacles,Crosswalk width,Vertical signs,Crossing length,Access to people with disabilities,Broken public property,Incivility,Obstacles,Physical aspects in poor condition,Poor infrastructure,Positive elements,Broken drain-cover,Drug addicts,Federal Judiciary,Panic button,Vandalism,graffiti,homelessness,inclined ramp,indication of security,mobile vendors,police presence,risky building,risky pavement,risky road,tourist map,trash,vehicle abandoned,excellent street lighting,good street lighting,regular street lighting,Coffee Shop,Food joint,Ice Cream Shop,Lighting Stores,Restaurant,Open-Weekdays,Open-Weekends,fully painted crosswalk,not painted crosswalk,partially painted crosswalk,Public WiFi,number of bikes,Mean Inhabitants,Employment,Crime Degree,school_count,Flooding,hospitals_count,Deaths,Injuries,Collision,Run over,Metro,Metrobus,Trolleybus,RTP,Concessioned,e-bike,Total coverage of transport
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,1.0,0.833333,0.666667,1.0,1.0,0.0,0.666667,0.333333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,0.125,0.0,0.59623,0.25,0.2,0.238095,5.0,0.247059,0.0,0.2,0.0,0.5,1.0,1.0,0.0,1.0,1.0,0.5,0.75


## Transforming to Maximization Criteria

The direction of preference (i.e., whether higher or lower values are better) determines whether a criterion is a maximization or minimization criterion. We will align all criteria in a way such that higher values are always better, which simplifies further analysis.The minimization criteria were transformed to maximization criteria by applying a reversed scale min–max normalization. The maximization criteria were normalized using a standard min–max normalization.

In [22]:
df_normalized.columns

Index(['Storefront', 'Visible Sign of Business', 'Business WiFi',
       'Business tables', 'Business Plugs', 'Business Internal CCTV',
       'External CCTV', 'Number of female employees', 'Accepts credit card',
       'Incline', 'Width', 'Ramp conditions', 'Ramp // Crosswalk',
       'Ramp obstacles', 'Crosswalk width', 'Vertical signs',
       'Crossing length', 'Access to people with disabilities',
       'Broken public property', 'Incivility', 'Obstacles',
       'Physical aspects in poor condition', 'Poor infrastructure',
       'Positive elements', 'Broken drain-cover', 'Drug addicts',
       'Federal Judiciary', 'Panic button', 'Vandalism', 'graffiti',
       'homelessness', 'inclined ramp', 'indication of security',
       'mobile vendors', 'police presence', 'risky building', 'risky pavement',
       'risky road', 'tourist map', 'trash', 'vehicle abandoned',
       'excellent street lighting', 'good street lighting',
       'regular street lighting', 'Coffee Shop', 'Food join

In [23]:
# Reverse the scale of the minimization criteria by subtracting from 1
minimization_criteria = ['Crime Degree', 'Broken public property', 'homelessness',
                         'risky road','Physical aspects in poor condition', 
                         'Incivility', 'Obstacles','risky building', 'risky pavement',
                        'police presence','Ramp obstacles','Poor infrastructure',
                        'Drug addicts','trash', 'vehicle abandoned','not painted crosswalk',
                        'Flooding','Deaths', 'Injuries', 'Collision','Run over',]

df_normalized[minimization_criteria] = 1 - df_normalized[minimization_criteria]

In [27]:
# initialize an empty list to store the percentile values
perc90 = []

# iterate over the columns in the DataFrame
for column in df_normalized.columns:
    # calculate the 90th percentile for the current column
    percentile_value = np.percentile(df_normalized[column], 90)
    # append the percentile value to the list
    perc90.append(percentile_value)

# convert the list of percentiles to a numpy array
perc90 = np.array(perc90)

perc90

array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.36      ,  0.        ,  0.5       ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.75      ,
        0.8       ,  0.        ,  0.2       ,  0.4       ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  0.1875    ,
        0.91666667,  0.67069768,  0.875     ,  1.        ,  0.23809524,
       -4.        ,  0.12156863,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  1.  

In [28]:
# initialize an empty list to store the percentile values
perc60 = []

# iterate over the columns in the DataFrame
for column in df_normalized.columns:
    # calculate the 90th percentile for the current column
    percentile_value = np.percentile(df_normalized[column], 60)
    # append the percentile value to the list
    perc60.append(percentile_value)

# convert the list of percentiles to a numpy array
perc60 = np.array(perc60)

perc60

array([ 1.        ,  1.        ,  0.9047619 ,  1.        ,  1.        ,
        1.        ,  1.        ,  0.03      ,  0.        ,  0.5       ,
        0.83333333,  1.        ,  1.        ,  1.        ,  0.57142857,
        0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.57142857,  1.        ,  0.        ,  0.125     ,
        0.72222222,  0.65545157,  0.75      ,  1.        ,  0.07936508,
       -4.        ,  0.01960784,  1.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  1.        ,  0.  

In [29]:
# initialize an empty list to store the percentile values
perc30 = []

# iterate over the columns in the DataFrame
for column in df_normalized.columns:
    # calculate the 90th percentile for the current column
    percentile_value = np.percentile(df_normalized[column], 30)
    # append the percentile value to the list
    perc30.append(percentile_value)

# convert the list of percentiles to a numpy array
perc30 = np.array(perc30)

perc30

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.125     ,
        0.54166667,  0.63628205,  0.5       ,  0.2       ,  0.03174603,
       -4.        ,  0.00784314,  1.        ,  0.8       ,  1.        ,
        0.66666667,  1.        ,  1.        ,  0.        ,  0.  